# comparing different approaches to `Distributions.py` with `numba`

In [1]:
import distcan as dc

from numba import jitclass, jit, vectorize
from numba import float32

import numpy as np
import timeit

import _rmath_ffi
from numba import cffi_support

from univariate import Normal, Chisq

cffi_support.register_module(_rmath_ffi)

## baseline `distcan` package

In [2]:
x = 2
y = np.linspace(-2,2,20)

In [3]:
N_dc = dc.Normal(0,1)

print("distcan -- pdf evaluated at one point:")
%timeit N_dc.pdf(x)

print("distcan -- pdf evaluated in a vector:")
%timeit N_dc.pdf(y)

distcan -- pdf evaluated at one point:
The slowest run took 5.52 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 146 µs per loop
distcan -- pdf evaluated in a vector:
10000 loops, best of 3: 140 µs per loop


## top level `@jit` and `@vectorize` with `rmath` functions

In [4]:
dnorm = _rmath_ffi.lib.dnorm

In [5]:
@jit(nopython=True)
def pdf_jit(mu, sigma, x):
    """The pdf value(s) evaluated at x."""
    return dnorm(x, mu, sigma, 0)

print("pure @jitted pdf evaluated at one point:")
%timeit pdf_jit(0, 1, x)

pure @jitted pdf evaluated at one point:
The slowest run took 355055.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 912 ns per loop


In [6]:
@vectorize(nopython=True)
def pdf_vect(mu, sigma, x):
    """The pdf value(s) evaluated at x."""
    return dnorm(x, mu, sigma, 0)

print("pure @vectorized pdf evaluated at a vector:")
%timeit pdf_vect(0, 1, x)

pure @vectorized pdf evaluated at a vector:


/home/daniel/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:257: RuntimeWarning: divide by zero encountered in pdf_vect


The slowest run took 10587.56 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.49 µs per loop


## `@jitclass`

In [7]:
spec = [
    ('mu', float32),
    ('sigma', float32),
]

@jitclass(spec)
class Normal_jitclass():
    """
    The Normal distribution with mean mu and standard deviation sigma.

    Parameters
    ----------
    mu : scalar(float)
        Mean of the normal distribution
    sigma : scalar(float)
        Standard deviaton of the normal distribution
    """

    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma

    def pdf(self, x):
        """The pdf value(s) evaluated at x."""
        return dnorm(x, self.mu, self.sigma, 0)
    
    def pdf_v(self, x):
        """The pdf value(s) evaluated at x."""
        y = np.empty_like(x)
        for k in range(x.shape[0]):
            y[k] = dnorm(x[k], self.mu, self.sigma, 0)
         
        return y

N_jc = Normal_jitclass(0,1)
print("@jitclass pdf evaluated at one point:")
%timeit N_jc.pdf(x)
print("@jitclass pdf evaluated in a vector:")
%timeit N_jc.pdf_v(y)

@jitclass pdf evaluated at one point:
The slowest run took 50797.47 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.54 µs per loop
@jitclass pdf evaluated in a vector:
The slowest run took 18755.05 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 12.2 µs per loop


## `@jitclass` calling top level `rmath`

In [8]:
spec = [
    ('mu', float32),
    ('sigma', float32),
]

@jitclass(spec)
class Normal_jitclass_top():
    """
    The Normal distribution with mean mu and standard deviation sigma.

    Parameters
    ----------
    mu : scalar(float)
        Mean of the normal distribution
    sigma : scalar(float)
        Standard deviaton of the normal distribution
    """

    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma

    def pdf(self, x):
        """The pdf value(s) evaluated at x."""
        return pdf_jit(self.mu, self.sigma, x)
    
    def pdf_v(self, x):
        """The pdf value(s) evaluated at x."""
        return pdf_vect(self.mu, self.sigma, x)

N_jc_top = Normal_jitclass_top(0,1)
print("@jitclass calling top level -- pdf evaluated at one point:")
%timeit N_jc_top.pdf(x)
print("@jitclass calling top level -- pdf evaluated in a vector:")
%timeit N_jc_top.pdf_v(y)
print("@jitclass calling top level -- pdf evaluated at a point with @vectorize:")
%timeit N_jc_top.pdf_v(x)

@jitclass calling top level -- pdf evaluated at one point:
The slowest run took 80412.16 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2 µs per loop
@jitclass calling top level -- pdf evaluated in a vector:
The slowest run took 33578.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 8.59 µs per loop
@jitclass calling top level -- pdf evaluated at a point with @vectorize:
The slowest run took 65705.31 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 1.95 µs per loop


## comparison in table

In [9]:
import warnings
warnings.filterwarnings("ignore")

print("=================================== distcan ===================================")
print("distcan -- pdf evaluated at one point:")
%timeit N_dc.pdf(x)

print("\ndistcan -- pdf evaluated in a vector:")
%timeit N_dc.pdf(y)

print("\n============================== @jit, @vectorize ===============================")
print("pure @jitted pdf evaluated at one point:")
%timeit pdf_jit(0, 1, x)
print("\npure @vectorized pdf evaluated at a vector:")
%timeit pdf_vect(0, 1, x)

print("\n================================== @jitclass ==================================")
print("@jitclass pdf evaluated at one point:")
%timeit N_jc.pdf(x)
print("\n@jitclass pdf evaluated in a vector:")
%timeit N_jc.pdf_v(y)

print("\n============================ @jitclass top level ==============================")
print("@jitclass calling top level -- pdf evaluated at one point:")
%timeit N_jc_top.pdf(x)
print("\n@jitclass calling top level -- pdf evaluated in a vector:")
%timeit N_jc_top.pdf_v(y)
print("@jitclass calling top level -- pdf evaluated at a point with @vectorize:")
%timeit N_jc_top.pdf_v(x)

=================================== distcan ===================================
distcan -- pdf evaluated at one point:
10000 loops, best of 3: 150 µs per loop

distcan -- pdf evaluated in a vector:
10000 loops, best of 3: 144 µs per loop

============================== @jit, @vectorize ===============================
pure @jitted pdf evaluated at one point:
The slowest run took 16.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 482 ns per loop

pure @vectorized pdf evaluated at a vector:
The slowest run took 10.74 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.55 µs per loop

================================== @jitclass ==================================
@jitclass pdf evaluated at one point:
The slowest run took 6.97 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.97 µs pe

# test for calling distributions from routines using numba

In [8]:
@jit(nopython=True)
def testN(x):
    """test"""
    N = Normal(0,1)
    return x + N.rand(1)

@jit(nopython=True)
def testC(x):
    """test"""
    C = Chisq(5)
    return x + C.rand(1)

print(testN(10))

print(testC(10))

[ 9.69862558]
[ 13.11184963]


In [14]:
@vectorize(nopython=True)
def testN2(x):
    """test"""
    N = Normal(0,1)
    return x + N.entropy


@vectorize(nopython=True)
def testC2(x):
    """test"""
    C = Chisq(5)
    return x + C.entropy

In [13]:
testN2(np.linspace(1,10,10))

array([  2.41893853,   3.41893853,   4.41893853,   5.41893853,
         6.41893853,   7.41893853,   8.41893853,   9.41893853,
        10.41893853,  11.41893853])

In [15]:
testC2(np.linspace(1,10,10))

array([  3.42309509,   4.42309509,   5.42309509,   6.42309509,
         7.42309509,   8.42309509,   9.42309509,  10.42309509,
        11.42309509,  12.42309509])

## take instance of distribution as argument

In [16]:
@jit(nopython=True)
def test_arg(dist, x):
    return x + dist.pdf(0)

In [17]:
N = Normal(0,1)

In [18]:
test_arg(N, 0)

0.3989422804014327

### does not work with `@vectorize`

In [19]:
@vectorize(nopython=True)
def test_arg(dist, x):
    return x + dist.pdf(0)